# <span style="color:gray">ipyrad-analysis toolkit:</span> abba-baba

The `baba` tool can be used to measure abba-baba statistics across many different hypotheses on a tree, to easily group individuals into populations for measuring abba-baba using allele frequencies, and to summarize or plot the results of many analyses. 

### Load packages

In [1]:
# conda install ipyrad -c conda-forge -c bioconda
# conda install ipcoal -c conda-forge

In [2]:
import ipyrad.analysis as ipa
import ipcoal
import toytree
import toyplot

In [3]:
print(ipa.__version__)
print(ipcoal.__version__)
print(toyplot.__version__)
print(toytree.__version__)

0.9.57
0.1.4
0.19.0
2.0.3


### Simulate linked SNP data from this tree/network
Here we simulate data on a tree/network that includes an introgressive edge. To approximate RAD-seq data we will also make ~50% of the data missing to approximate dropout from mutation disruption or insufficient sequencing coverage. 

In [4]:
# generate a balance tree
tree = toytree.rtree.baltree(ntips=10, treeheight=10e6)

# draw the tree w/ an admixture edge
tree.draw(ts='p', admixture_edges=(2, 3));

<svg class="toyplot-canvas-Canvas" height="260.0px" id="t22fd9a0f057f4e9b974c415e523c4f3d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 0 5000000 10000000

In [5]:
# create a simulation model for this tree/network: (src, dest, time-prop., admix-prop.)
model = ipcoal.Model(tree=tree, nsamples=2, Ne=4e5, admixture_edges=(2, 3, 0.5, 0.15))

# simulate N loci
model.sim_loci(nloci=3000, nsites=50)

# drop 50% as missing
model.apply_missing_mask(0.5)

# write result to a database file
model.write_snps_to_hdf5(name="test-baba-miss50", outdir="/tmp", diploid=True)

wrote 64626 SNPs to /tmp/test-baba-miss50.snps.hdf5


### Input data for BABA tests
We need a SNPs database and we need to know the sample names that in that database. As we will show, one easy way to generate tests for a dataset is to use a tree that has the sample labels on the tips (like we created above.) This is optional though, you can also simply write out the tests by hand. 

In [5]:
# init a baba tool from your SNPs database
tool = ipa.baba2("/tmp/test-baba-miss50.snps.hdf5")

### The format of test hypotheses
The simplest way to describe a test is using a dictionary with keys labeled as "p1", "p2", "p3", "p4", and the values listing the tip names that will be used to represent a population/species. When multiple samples are listed the pooled allele frequency will be used. 

In [6]:
IMAP1 = {
    "p4": ["r7", "r6", "r5"],
    "p3": ["r3"],
    "p2": ["r2"],
    "p1": ["r0", "r1"],
}

IMAP2 = {
    "p4": ["r7", "r6", "r5"],
    "p3": ["r3"],
    "p2": ["r1"],
    "p1": ["r0"],
}

### A simple tests
Here we will run the first tests (IMAP1) which includes a scenario where we expect to see evidence of admixture since our simulation included gene flow from P3 to P2 backward in time. It is good to start with simple tests before proceeding to run many tests in parallel. The output from a single test is verbose about how SNP filtering is applied, and how many total loci, SNPs, and discordant SNPs are present. 

Running an ABBA-BABA test requires that there is data for at least one sample in each population. But when you have multiple samples per population you can also apply further filtering using a minmap argument. For example, a minmap with values set to 0.75 would require that data is present for 75% of samples in a population for the SNP to be included in analyses. The verbose output below will print how many SNPs are filtered by the minmap and other filters.

In [7]:
# run a single test 
tool.run_test(IMAP1)

Samples: 7
Sites before filtering: 64626
Filtered (indels): 0
Filtered (bi-allel): 8096
Filtered (mincov): 3565
Filtered (minmap): 29365
Filtered (subsample invariant): 15099
Filtered (combined): 41455
Sites after filtering: 23171
Sites containing missing values: 16291 (70.31%)
Missing values in SNP matrix: 24400 (15.04%)


,D,bootstd,Z,ABBA,BABA,nSNPs,nloci
0,0.533,0.032,16.776,551.667,167.833,23171,1566


In [8]:
# run the same test with different filtering applied
tool.run_test(imap=IMAP1, minmap={i: 0.5 for i in IMAP1})

Samples: 7
Sites before filtering: 64626
Filtered (indels): 0
Filtered (bi-allel): 8096
Filtered (mincov): 3565
Filtered (minmap): 34239
Filtered (subsample invariant): 15099
Filtered (combined): 44594
Sites after filtering: 20032
Sites containing missing values: 13152 (65.65%)
Missing values in SNP matrix: 16866 (12.03%)


,D,bootstd,Z,ABBA,BABA,nSNPs,nloci
0,0.54,0.037,14.706,457.667,136.833,20032,1334


### Running multiple tests (simple)
You may wish to run many tests and collect the results in a table. This can be automated and run in parallel using the `.run()` argument. The tests are submitted as a list of dictionaries. 

In [9]:
# enter multiple tests (imaps) and multiple minmaps to run in parallel
tool.run(imaps=[IMAP1, IMAP2])

[####################] 100% 0:00:12 | abba-baba tests 


In [10]:
# show the results
tool.results_table

,D,bootstd,Z,ABBA,BABA,nSNPs,nloci
0,0.533,0.034,15.715,551.667,167.833,23171,1566
1,0.173,0.066,2.640,103.583,73.042,17649,1280


### Running multiple tests (automated)
Using a tree you can conveniently describe a large set of hypotheses to test that are all compatible with the phylogenetic topology. The list of tests can be further added to by hand if you want to also include tests that are incompatible with the phylogeny. Multiple sample names can be selected by entering the idx node label from the tree drawing (see below).

In [11]:
tree.draw(ts='p', admixture_edges=(2, 3));

<svg class="toyplot-canvas-Canvas" height="260.0px" id="tb34e2f2975764d5683c6f645755880dc" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 0 5000000 10000000

The function `.generate_tests_from_tree()` takes the tree as an argument in addition to a `constraint_dict` and a `constraint_exact` argument. This tool will then generate a list of all tests that are compatible with the tree given the constraints put on it. For example, (None, None, None, 13) would only put the constraint that the "p4" taxon must descend from node 13. The `constraint_exact=1` on node 13 means that tests must include *all* descendants of node 13. By contrast, "p3" is descendant of node 10 but has `constraint_exact=0` meaning it can be node 0, 1, 2, 3, 8, 9 or 10. Still the "p3" options will be constrained by the topology such that the sister lineage must include at least two tips. This leads to 10 possible tests below, shown by their idx node labels in the order p1-4.

In [12]:
# generate a list with tests from several constraints
tests1 = tool.generate_tests_from_tree(
    tree=tree, 
    constraint_dict=(None, None, 13, 17), 
    constraint_exact=(0, 0, 0, 1),
)

12 tests generated from tree


In [13]:
# generate a list with tests from several constraints
tests2 = tool.generate_tests_from_tree(
    tree=tree, 
    constraint_dict=(None, None, 12, 17), 
    constraint_exact=(0, 0, 0, 1),
)

6 tests generated from tree


In [14]:
tests = tests1 + tests2

In [15]:
# # add more tests manually (tip: get tips desc from node idx using .get_tip_labels())
# tests.append({
#     'p1': tree.get_tip_labels(0),
#     'p2': ['r1', 'r3'], 
#     'p3': tree.get_tip_labels(4), 
#     'p4': tree.get_tip_labels(13), 
# })

In [16]:
# pass a list with tests to .run() and see results in .results_table
tool.run(tests)

[####################] 100% 0:01:08 | abba-baba tests 


### Interpreting results
The results can be viewed as a pandas DataFrame in (`.results_table`) and saved to a CSV file. The most relevant statistic in the Z score, which represents the number of standard deviations (measured by bootstrap resampling) that D deviates from zero. Values greater than 3 are generally considered significant, but you can lookup Z for given alpha significance level, and you should consider that there are many tests as well when deciding on significance. 

For ease of viewing (maybe) the taxa that are involved in each test are stored in a separate data frame (`.taxon_table`). You can concatenate these two dataframes together before saving if you wish, as you could with any pandas dataframes. 

In [17]:
# the results of tests run with .run are summarized in .results_table
tool.results_table

,D,bootstd,Z,ABBA,BABA,nSNPs,nloci
0,0.578,0.031,18.628,512.008,136.767,27793,1586
1,0.501,0.035,14.347,435.863,145.029,28830,1611
2,0.477,0.033,14.663,522.358,184.921,36163,1970
3,0.452,0.038,11.867,407.950,153.871,27918,1568
4,0.149,0.090,1.652,80.933,59.896,21482,1294
5,0.074,0.070,1.055,93.642,80.696,28301,1619
6,0.300,0.044,6.829,246.942,133.100,21186,1244
7,0.627,0.039,16.228,434.292,99.600,21984,1291
8,0.548,0.045,12.248,405.867,118.658,21583,1270
9,0.346,0.041,8.384,328.650,159.625,27644,1571


In [18]:
# the sample names in tests that were run are also summarized in .taxon_table
tool.taxon_table

,p1,p2,p3,p4
0,"r0,r1",r2,r3,"r5,r6,r7,r8,r9"
1,r0,r2,"r3,r4","r5,r6,r7,r8,r9"
2,"r0,r1",r2,"r3,r4","r5,r6,r7,r8,r9"
3,r1,r2,"r3,r4","r5,r6,r7,r8,r9"
4,r0,r1,r3,"r5,r6,r7,r8,r9"
5,r0,r1,"r3,r4","r5,r6,r7,r8,r9"
6,r1,r2,r4,"r5,r6,r7,r8,r9"
7,r0,r2,r3,"r5,r6,r7,r8,r9"
8,r1,r2,r3,"r5,r6,r7,r8,r9"
9,"r0,r1",r2,r4,"r5,r6,r7,r8,r9"


### Plotting results
The `.draw()` function takes a tree as input and will organize the results stored to the `baba` tool object into a more easily interpretable diagram. Here the p4, p3, p2, and p1 samples in each tests are indicated by different colors. The histogram on the left shows the distribution of bootstrap D-statistic values in each test and is colored to show the dominant pattern ABBA (orange) or BABA (green) if the test is significant at p=0.01 (Z > 2.5). The data used in this plot is from the `.results_table` and `.taxon_table` shown above.

In [21]:
# generate a summary drawing of test results
canvas = tool.draw(tree=tree, sort=True, width=500, height=500)

<svg class="toyplot-canvas-Canvas" height="500.0px" id="t750de1f576964d40ac652b2f7e69fb3a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 500.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Z-score: -0.38 Z-score: -0.1 Z-score: -2.35 Z-score: -6.13 Z-score: -1.65 Z-score: -16.23 Z-score: -12.25 Z-score: -18.63 Z-score: -0.78 Z-score: -7.27 Z-score: -6.83 Z-score: -8.38 Z-score: -0.13 Z-score: -2.98 Z-score: -1.05 Z-score: -14.35 Z-score: -11.87 Z-score: -14.66 -20 -10 0 Z-score <polygon points="25.0,124.4078947368421 27.93103448275862,124.4078947368421 30.862068965517242,124.4078947368421 33.79310344827586,124.4078947368421 36.724137931034484,124.4078947368421 39.6551724137931,124.4078947368421 42.58620689655172,124.4078947368421 45.51724137931034,124.4078947368421 48.44827586206897,124.4078947368421 51.37931034482759,124.4078947368421 54.31034482758621,124.4078947368421 57.241379310344826,124.4078947368421 60.172413793103445,124.4078947368421 63.103448275862064,124.4078947368421 66.03448275862068,124.4078947368421 68.96551724137932,124.4078947368421 71.89655172413794,124.4078947368421 74.82758620689656,124.4078947368421 77.75862068965517,124.4078947368421 80.6896551724138,124.4078947368421 83.62068965517243,124.4078947368421 86.55172413793105,124.4078947368421 89.48275862068965,124.4078947368421 92.41379310344827,124.4078947368421 95.34482758620689,124.4078947368421 98.27586206896551,124.4078947368421 101.20689655172414,124.4078947368421 104.13793103448275,124.4078947368421 107.06896551724137,124.4078947368421 110.0,124.4078947368421 110.0,124.4078947368421 107.06896551724137,124.4078947368421 104.13793103448275,124.4078947368421 101.20689655172414,124.4078947368421 98.27586206896551,124.4078947368421 95.34482758620689,124.4078947368421 92.41379310344827,124.4078947368421 89.48275862068965,124.4078947368421 86.55172413793105,124.40789473683989 83.62068965517243,124.40789473608046 80.6896551724138,124.40789459995236 77.75862068965517,124.40788187253824 74.82758620689656,124.40726258909169 71.89655172413794,124.3916516800689 68.96551724137932,124.18965502203841 66.03448275862068,122.87463169908378 63.103448275862064,118.77519754204098 60.172413793103445,113.58775172147209 57.241379310344826,113.53947368421055 54.31034482758621,118.69946359577101 51.37931034482759,122.84011901066933 48.44827586206897,124.18274680446967 45.51724137931034,124.39098764749568 42.58620689655172,124.40723086142066 39.6551724137931,124.407881106047 36.724137931034484,124.40789459049891 33.79310344827586,124.40789473602055 30.862068965517242,124.40789473683967 27.93103448275862,124.4078947368421 25.0,124.4078947368421" style="fill:rgb(70.2%,70.2%,70.2%);fill-opacity:1.0;opacity:1.0;stroke:rgb(0%,0%,0%);stroke-opacity:1.0;stroke-width:0.5"> D-statistic: 0.02 <polygon points="25.0,139.93421052631578 27.93103448275862,139.93421052631578 30.862068965517242,139.93421052631578 33.79310344827586,139.93421052631578 36.724137931034484,139.93421052631578 39.6551724137931,139.93421052631578 42.58620689655172,139.93421052631578 45.51724137931034,139.93421052631578 48.44827586206897,139.93421052631578 51.37931034482759,139.93421052631578 54.31034482758621,139.93421052631578 57.241379310344826,139.93421052631578 60.172413793103445,139.93421052631578 63.103448275862064,139.93421052631578 66.03448275862068,139.93421052631578 68.96551724137932,139.93421052631578 71.89655172413794,139.93421052631578 74.82758620689656,139.93421052631578 77.75862068965517,139.93421052631578 80.6896551724138,139.93421052631578 83.62068965517243,139.93421052631578 86.55172413793105,139.93421052631578 89.48275862068965,139.93421052631578 92.4

In [22]:
# save the drawing to a PDF
import toyplot.pdf
toyplot.pdf.render(canvas, "/tmp/BABA-plot.pdf")

### Non-independence of D-statistics
While ABBA-BABA tests are a powerful tool for testing hypotheses about admixture, they are seriously prone to false positives when interpreting results. This is because each 4-taxon test is non-independent of other tests that involve related taxa ([Eaton et al. 2015](https://onlinelibrary.wiley.com/doi/10.1111/evo.12758)). 

This is clear in the example above, where the true introgressive pattern was "r3" into "r2" (forward in time). The results show admixture between these samples (e.g., tests 3, 5, 6, 7, 13 above), however, they also pick up a signal of introgression between "r4" and "r2" (e.g., tests 9,10, 11). This is because 'r3' shares ancestry with 'r4', and so it passed alleles into 'r2' that it shares with 'r4'. Ideally we would want to be able to distinguish whether 'r4' and 'r3' both introgressed into 'r2' independently, or whether it is just a signal of their shared ancestry. This is the goal of "partitioned D-statistics" (Eaton et al. 2013, 2015). 

### Running 5-taxon (partitioned) D-statistics
To run a 5-taxon partitioned D-statistic test you must define tests using a dictionary that includes two clades sampled as potential introgressive donors in the P3 clade. These are labeled `p3_1` and `p3_2`. The partitioned test then returns three D-statistics: D12 represents introgression of alleles shared by the two P3 sublineages, D1 is alleles that uniquely introgressed from P3_1 and D2 is alleles that uniquely introgressed from P3_2. The D12 statistic is particularly powerful since it indicates the direction of introgression. **If a test does not have a significant D12 then introgression from that P3 lineage is not supported**. You should try testing in the opposite direction, as in the example below. 


In [23]:
tool = ipa.baba2("/tmp/test-baba-miss50.snps.hdf5")

In [24]:
# testing r3 vs r4 as an introgressive donor into r2 or r0,r1
IMAP5 = {
    "p1": ["r0", "r1"],
    "p2": ["r2"],
    "p3_1": ["r3"],
    "p3_2": ["r4"],
    "p4": ['r5', 'r6', 'r7'],
}

# the reverse test of 'r2' versus 'r0,r1' as introgressive donors into r3 or r4
IMAP5_REV = {
    "p1": ["r3"],
    "p2": ["r4"],
    "p3_1": ["r0", "r1"],
    "p3_2": ["r2"],
    "p4": ['r5', 'r6', 'r7'],
}

### Partitioned D-statistic results
The first test (IMAP5) has a significant Z12 with many more ABBBA patterns than BABBA, supporting P3 as an introgressive donor into P2. Of the two P3 lineages, we see that there is significant support for introgression of P3_1 into P2 (Z1) but not of P3_2 into P2 (Z2). Thus we know that introgression occurred from P3_1 and not P3_2 into P2. 

In [25]:
# significant Z12 and Z1 
tool.run_partitioned_test(IMAP5, quiet=True)

,D12,D1,D2,boot12std,boot1std,boot2std,Z12,Z1,Z2,ABBBA,BABBA,ABBAA,BABAA,ABABA,BAABA,nSNPs,nloci
0,0.413,0.621,0.081,0.048,0.055,0.084,8.63,11.198,0.964,212.0,88.167,179.417,42.0,59.583,50.667,19067,1173


The ability for partitioned D-statistics to show the direction of introgression is clearly demonstrated in the reverse test. We know that in our simulation introgression occured from `r3` into `r2` (forward in time), but the test below (IMAP5_REV) is asking about the reverse pattern. As we saw above, the 4-taxon tests were not able to clearly distinguish the direction of introgression. However, here we can see that there *not a significant Z12 statistic*, meaning that this direction of introgression is not supported. The significant Z2 statistic shows that the P3_2 taxon shares many alleles with P2 (there is admixture) but the lack of significant Z12 means it is not supported as an introgressive donor.

In [26]:
# non-significant Z12 means that the hypothesis is not supported
tool.run_partitioned_test(IMAP5_REV, quiet=True)

,D12,D1,D2,boot12std,boot1std,boot2std,Z12,Z1,Z2,ABBBA,BABBA,ABBAA,BABAA,ABABA,BAABA,nSNPs,nloci
0,0.021,0.094,-0.501,0.064,0.093,0.056,0.322,1.008,9.033,104.5,100.25,50.667,42.0,59.583,179.417,19067,1173


### Partitioned D-tests versus Dfoil...
You may have seen a paper in 2015 criticizing the partitioned D-statistics and presenting a modification to this method named D-foil. I do not agree with the criticism in this paper of the partitioned tests. The paper suffered major problems with its simulation framework (e.g., ms simulations with gene flow in the wrong direction, migration rates >50%, etc.) that made the critique of partitioned D-tests unrealistic and artificial, in my opinion.